In [1]:
from brightway2 import *

In [3]:
projects.set_current("Test") # with ecoinvent database loaded

In [5]:
db = Database("cutoff38")

In [12]:
process_list = db.search('electricity, high voltage, production mix', limit=10000000)

In [39]:
process = [p for p in process_list if 'electricity, high voltage, production mix' in str(p) and 'CA-SK' in str(p)][0]

In [40]:
process

'electricity, high voltage, production mix' (kilowatt hour, CA-SK, None)

In [42]:
functional_unit = {process: 1}

In [43]:
CC_method = [m for m in methods if 'IPCC 2013' in str(m) and  'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]

In [44]:
lca = LCA(functional_unit, CC_method)

In [46]:
lca.lci()
lca.lcia()

In [47]:
lca.demand

{'electricity, high voltage, production mix' (kilowatt hour, CA-SK, None): 1}

In [48]:
lca.method

('IPCC 2013', 'climate change', 'GWP 100a')

In [49]:
lca.score

0.7205447304969017